In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://hoopshype.com/salaries/players/'
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://hoopshype.com/salaries/players/2021-2022',
 'https://hoopshype.com/salaries/players/2020-2021',
 'https://hoopshype.com/salaries/players/2019-2020',
 'https://hoopshype.com/salaries/players/2018-2019',
 'https://hoopshype.com/salaries/players/2017-2018']

In [4]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:

    # provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Make python pause for some time
    sleep(2)

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table', class_='hh-salaries-ranking-table hh-salaries-table-sortable responsive')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('td')[1:4]:
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Create a dataframe using the headers as column names
    p_salary = pd.DataFrame(columns = headers) 
    
    # Create a for loop to fill p_salary
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "").strip() for j in row_data] 
        length = len(p_salary)
        p_salary.loc[length] = row

    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(p_salary)


Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Completed: 24 webpage(s) scraped.
Completed: 25 webpage(s) scraped.
Completed: 26 webpage(s) scraped.


In [5]:
dataframe_list

[                Player      2021/22   2021/22(*)
 0        Stephen Curry  $45,780,966  $45,780,966
 1         James Harden  $44,310,840  $44,310,840
 2            John Wall  $44,310,840  $44,310,840
 3    Russell Westbrook  $44,211,146  $44,211,146
 4         Kevin Durant  $42,018,900  $42,018,900
 ..                 ...          ...          ...
 648     Anthony Barber      $53,176      $53,176
 649       Luca Vildoza      $42,789      $42,789
 650     Zavier Simpson      $37,223      $37,223
 651  Mfiondu Kabengele      $19,186      $19,186
 652     Melvin Frazier      $13,294      $13,294
 
 [653 rows x 3 columns],
                 Player      2020/21   2020/21(*)
 0        Stephen Curry  $43,006,362  $45,325,029
 1    Russell Westbrook  $41,358,814  $43,588,654
 2           Chris Paul  $41,358,814  $43,588,654
 3            John Wall  $41,254,920  $43,479,158
 4         James Harden  $41,254,920  $43,479,158
 ..                 ...          ...          ...
 573       Cam Reynolds

In [11]:
from functools import reduce
test = reduce(lambda l, r: pd.merge(l, r, on='Player', how='left'), dataframe_list)
test

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$30,570,000","$32,960,751","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$19,169,800","$20,668,989","$18,063,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$44,211,146","$41,358,814","$43,588,654","$38,506,482","$40,844,595","$35,665,000","$38,454,209","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,018,900","$40,108,950","$42,271,404","$37,199,000","$39,457,722","$30,000,000","$32,346,173","$25,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,Anthony Barber,"$53,176","$53,176",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
649,Luca Vildoza,"$42,789","$42,789","$3,500,000","$3,688,700",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,Zavier Simpson,"$37,223","$37,223",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651,Mfiondu Kabengele,"$19,186","$19,186","$2,432,353","$2,563,492","$1,977,000","$2,097,043",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
curry = test[test['Player'] == 'Stephen Curry']
curry

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
nba_stats = pd.read_csv('nba_stats.csv', encoding='utf-8')
nba_stats

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,500,553,90.4,50,234,284,737,303,72,7
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,520,593,87.7,56,336,392,374,181,68,24
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,654,803,81.4,146,650,796,284,214,77,99
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,400,469,85.3,85,524,609,334,217,75,49
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,379,468,81.0,206,813,1019,584,281,109,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,25,1,0,1,14,0,0,5,0.0,...,0,0,0.0,2,5,7,0,0,0,0
12303,Bruce Bowen,26,1,1,0,1,0,0,0,0.0,...,0,0,0.0,0,0,0,0,0,0,1
12304,Cuonzo Martin,25,3,0,3,13,0,0,7,0.0,...,0,0,0.0,1,0,1,1,1,0,0
12305,Derrick Alston,24,2,1,1,11,0,0,5,0.0,...,0,2,0.0,3,1,4,0,0,0,0


In [ ]:
test = dataframe_list[0].append(dataframe_list[1], ignore_index=True)
test[650:700]

In [ ]:
player1 = salaries[:][salaries['Player'].notna()]
player1

In [ ]:
test = dataframe_list[0][dataframe_list[0]['Player'] == 'Stephen Curry']
test

In [14]:
test2 = pd.merge(nba_stats, test, left_on="PLAYER", right_on="Player")
test2

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Trae Young,22,63,36,27,2125,1594,487,1112,43.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trae Young,21,60,18,42,2120,1778,546,1249,43.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Trae Young,20,81,29,52,2503,1549,525,1256,41.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008,Larry Sanders,26,27,12,15,586,196,86,172,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3009,Larry Sanders,25,23,3,20,584,178,76,162,46.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,Larry Sanders,24,71,34,37,1937,696,306,605,50.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,Larry Sanders,23,52,24,28,643,186,84,184,45.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test2 = dataframe_list[1][dataframe_list[1]['Player'] == 'Stephen Curry']
test2

In [ ]:
test3 = pd.merge(test, test2, on='Player', how='right')

In [ ]:
test3

In [ ]:
curry = pd.read_csv('curry.csv', encoding='utf-8', index_col=[0])

In [ ]:
curry

In [ ]:
curry_sal = pd.merge(curry, test3, left_on="PLAYER", right_on="Player")
curry_sal

In [ ]:
curry_sal2 = pd.merge(curry, test3.set_index("Player"), left_on="PLAYER", right_index=True)
curry_sal2